# Практическое занятие: Персептрон и простая полносвязная нейросеть (NumPy + Pandas + Plotly)

В этом блокноте мы шаг за шагом разберёмся с работой простого персептрона и небольшой полносвязной нейросети (1 скрытый слой), реализованными «с нуля» на NumPy. Будем использовать маленький датасет (≤5000 образцов) и визуализировать результаты с помощью Plotly.

## Чек-лист урока
- [x] Краткий ресерч небольших датасетов (≤5000 образцов)
- [x] Загрузка небольшого реального датасета через Pandas
- [x] EDA и базовая визуализация (Plotly)
- [x] Персептрон: формулы, обучение, визуализация границы
- [x] FCNN (1 скрытый слой): прямой ход, функции активации, потери, обратное распространение, обучение
- [x] Оценка качества и выводы




## Небольшие датасеты: что выбрать для практики

Ниже — небольшая подборка простых и широко используемых датасетов (все ≤5000 образцов), подходящих для занятия по персептрону/полносвязной НС. Ссылки на страницы и удобные CSV-источники:

- Iris (150 объектов, 3 класса, 4 признака)
  - Описание: `https://archive.ics.uci.edu/ml/datasets/iris`
  - CSV (готовый с заголовками): `https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv`
- Wine (178 объектов, 3 класса, 13 признаков)
  - Описание: `https://archive.ics.uci.edu/ml/datasets/wine`
  - RAW data (без заголовков): `https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data`
- Breast Cancer Wisconsin Diagnostic (569 объектов, 2 класса)
  - Описание: `https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)`
  - RAW data (без заголовков): `https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data`
- Pima Indians Diabetes (768 объектов, 2 класса)
  - Описание: `https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes`
  - CSV (с заголовками в community-версии): `https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv`
- Banknote Authentication (1372 объекта, 2 класса, 4 признака)
  - Описание: `https://archive.ics.uci.edu/ml/datasets/banknote+authentication`
  - CSV: `https://raw.githubusercontent.com/jbrownlee/Datasets/master/banknote_authentication.csv`

Дополнительно:
- OpenML (каталог датасетов): `https://www.openml.org`
- Наборы scikit-learn (описания): `https://scikit-learn.org/stable/datasets/toy_dataset.html`

В этом практикуме мы используем Iris как основной пример: он очень маленький, чистый и понятный.


In [ ]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from typing import Tuple

np.random.seed(42)

# Небольшие утилиты

def train_test_split_np(X: np.ndarray, y: np.ndarray, test_size: float = 0.2, shuffle: bool = True, seed: int = 42) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Простейший train/test split на NumPy без зависимостей."""
    assert 0 < test_size < 1
    n_samples = X.shape[0]
    indices = np.arange(n_samples)
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(indices)
    test_count = int(n_samples * test_size)
    test_idx = indices[:test_count]
    train_idx = indices[test_count:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]


def zscore_fit_transform(X: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Стандартизация z-score: возвращает X_scaled, mean, std."""
    mean = X.mean(axis=0, keepdims=True)
    std = X.std(axis=0, keepdims=True) + 1e-8
    return (X - mean) / std, mean, std


def zscore_transform(X: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
    return (X - mean) / (std + 1e-8)



## Загружаем датасет Iris и делаем краткий EDA

Источник CSV: `https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv`

Столбцы: `sepal_length`, `sepal_width`, `petal_length`, `petal_width`, `species`.


In [ ]:
iris_url = "https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv"
df = pd.read_csv(iris_url)
print(df.head())
print("\nФорма:", df.shape)
print("\nКлассы:")
print(df["species"].value_counts())

fig = px.scatter_matrix(
    df,
    dimensions=["sepal_length", "sepal_width", "petal_length", "petal_width"],
    color="species",
    title="Iris: Scatter Matrix по признакам"
)
fig.show()


## Персептрон: бинарная классификация двух видов Iris

Для персептрона выберем два класса (например, `setosa` и `versicolor`) и два признака, чтобы можно было красиво визуализировать границу решения. Признаки: `sepal_length`, `petal_length`.

Метки преобразуем в {-1, +1} для классического обновления весов.


In [ ]:
df_bin = df[df["species"].isin(["setosa", "versicolor"])].copy()
feat_cols = ["sepal_length", "petal_length"]
X = df_bin[feat_cols].to_numpy(dtype=float)
y = (df_bin["species"].to_numpy() == "versicolor").astype(int)
# Преобразуем метки в {-1, +1}
y = np.where(y == 1, 1, -1)

# Стандартизация только по train
X_train, X_test, y_train, y_test = train_test_split_np(X, y, test_size=0.2, shuffle=True, seed=42)
X_train_scaled, m, s = zscore_fit_transform(X_train)
X_test_scaled = zscore_transform(X_test, m, s)

print("Размеры:", X_train_scaled.shape, X_test_scaled.shape)

fig = px.scatter(
    x=X_train_scaled[:, 0], y=X_train_scaled[:, 1], color=y_train.astype(str),
    labels={"x": feat_cols[0], "y": feat_cols[1]},
    title="Train точки (стандартизованные)"
)
fig.show()


### Теория: обновление весов персептрона

- Активация: \(\hat{y} = \operatorname{sign}(w^T x + b)\)
- Ошибка: \(e = y - \hat{y}\)
- Обновление параметров (перцептронное правило): \(w \leftarrow w + \eta\, e\, x\), \(b \leftarrow b + \eta\, e\)

Где \(\eta\) — скорость обучения. Для бинарной классификации удобно \(y \in \{-1, +1\}\).

Ссылки:
- Персептрон (RU): `https://ru.wikipedia.org/wiki/Персептрон`


In [ ]:
class Perceptron:
    def __init__(self, learning_rate: float = 0.1, epochs: int = 50):
        self.learning_rate = float(learning_rate)
        self.epochs = int(epochs)
        self.w = None  # shape: (n_features,)
        self.b = 0.0

    def fit(self, X: np.ndarray, y: np.ndarray):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features, dtype=float)
        self.b = 0.0
        self.errors_per_epoch = []

        for _ in range(self.epochs):
            errors = 0
            for xi, yi in zip(X, y):
                linear = np.dot(xi, self.w) + self.b
                y_hat = 1 if linear >= 0 else -1
                err = yi - y_hat
                if err != 0:
                    self.w += self.learning_rate * err * xi
                    self.b += self.learning_rate * err
                    errors += 1
            self.errors_per_epoch.append(errors)
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        linear = X @ self.w + self.b
        return np.where(linear >= 0, 1, -1)


In [ ]:
per_clf = Perceptron(learning_rate=0.1, epochs=50)
per_clf.fit(X_train_scaled, y_train)

fig = px.line(y=per_clf.errors_per_epoch, labels={"x": "Эпоха", "y": "Ошибок за эпоху"}, title="Сходимость персептрона")
fig.show()

y_pred = per_clf.predict(X_test_scaled)
acc = (y_pred == y_test).mean()
print(f"Точность (test): {acc:.3f}")


In [ ]:
# Визуализация границы решения (2D)
xx, yy = np.meshgrid(
    np.linspace(X_train_scaled[:, 0].min()-0.5, X_train_scaled[:, 0].max()+0.5, 200),
    np.linspace(X_train_scaled[:, 1].min()-0.5, X_train_scaled[:, 1].max()+0.5, 200)
)
grid = np.c_[xx.ravel(), yy.ravel()]
zz = per_clf.predict(grid).reshape(xx.shape)

fig = go.Figure()
fig.add_trace(go.Contour(
    x=xx[0], y=yy[:, 0], z=zz,
    showscale=False, opacity=0.4, colorscale=[[0, '#A1D99B'], [1, '#9ECAE1']],
    contours=dict(coloring='heatmap', showlabels=False)
))

fig.add_trace(go.Scatter(
    x=X_train_scaled[:, 0], y=X_train_scaled[:, 1],
    mode='markers',
    marker=dict(color=y_train, colorscale=[[0, 'red'], [1, 'blue']]),
    name='Train'
))

fig.update_layout(title='Персептрон: граница решения', xaxis_title=feat_cols[0], yaxis_title=feat_cols[1])
fig.show()


## Полносвязная нейросеть (1 скрытый слой) с нуля на NumPy

Далее реализуем минимальную FCNN для многоклассовой классификации на Iris. Возьмём 2 признака — `petal_length`, `petal_width`, т.к. они хорошо разделяют классы визуально. Выход — 3 класса (softmax), функция потерь — кросс-энтропия.

### Формулы (кратко)
- Прямой ход:
  - \(Z_1 = X W_1 + b_1\), \(A_1 = \phi(Z_1)\) (возьмём ReLU)
  - \(Z_2 = A_1 W_2 + b_2\), \(\hat{Y} = \operatorname{softmax}(Z_2)\)
- Потери (кросс-энтропия): \(L = -\frac{1}{N} \sum_i \sum_k y_{ik} \log(\hat{y}_{ik})\)
- Обратное распространение:
  - \(\nabla_{Z_2} = \hat{Y} - Y\)
  - \(\nabla_{W_2} = A_1^T \nabla_{Z_2} / N\), \(\nabla_{b_2} = \sum_i \nabla_{Z_2}^{(i)} / N\)
  - \(\nabla_{A_1} = \nabla_{Z_2} W_2^T\), \(\nabla_{Z_1} = \nabla_{A_1} \odot \phi'(Z_1)\)
  - \(\nabla_{W_1} = X^T \nabla_{Z_1} / N\), \(\nabla_{b_1} = \sum_i \nabla_{Z_1}^{(i)} / N\)

Ссылки:
- Обратное распространение (RU): `https://ru.wikipedia.org/wiki/Обратное_распространение_ошибки`
- CS231n Backprop: `https://cs231n.github.io/optimization-2/`
- Michael Nielsen: `http://neuralnetworksanddeeplearning.com/chap1.html`


In [ ]:
# Готовим данные для многоклассовой задачи
feat_mlp = ["petal_length", "petal_width"]
X_full = df[feat_mlp].to_numpy(dtype=float)
# Преобразуем species в 0/1/2
label_to_id = {lbl: i for i, lbl in enumerate(sorted(df["species"].unique()))}
y_full = df["species"].map(label_to_id).to_numpy(dtype=int)

X_train2, X_test2, y_train2, y_test2 = train_test_split_np(X_full, y_full, test_size=0.2, shuffle=True, seed=42)
X_train2_s, mean2, std2 = zscore_fit_transform(X_train2)
X_test2_s = zscore_transform(X_test2, mean2, std2)

num_classes = len(np.unique(y_full))
print("Классов:", num_classes, label_to_id)


In [ ]:
# Функции активации и служебные функции

def relu(x: np.ndarray) -> np.ndarray:
    return np.maximum(0.0, x)

def relu_grad(x: np.ndarray) -> np.ndarray:
    return (x > 0).astype(float)

def softmax(z: np.ndarray) -> np.ndarray:
    z_shift = z - z.max(axis=1, keepdims=True)
    exp_z = np.exp(z_shift)
    return exp_z / (exp_z.sum(axis=1, keepdims=True) + 1e-12)

def one_hot(y: np.ndarray, num_classes: int) -> np.ndarray:
    oh = np.zeros((y.shape[0], num_classes), dtype=float)
    oh[np.arange(y.shape[0]), y] = 1.0
    return oh

def cross_entropy(y_true_oh: np.ndarray, y_pred_proba: np.ndarray) -> float:
    # средняя кросс-энтропия
    return float(-np.mean(np.sum(y_true_oh * np.log(y_pred_proba + 1e-12), axis=1)))


In [ ]:
class MLP:
    def __init__(self, in_features: int, hidden_units: int, out_classes: int, lr: float = 0.05, seed: int = 42):
        self.in_features = int(in_features)
        self.hidden_units = int(hidden_units)
        self.out_classes = int(out_classes)
        self.lr = float(lr)
        rng = np.random.default_rng(seed)
        # He init для ReLU
        self.W1 = rng.normal(0.0, np.sqrt(2.0 / in_features), size=(in_features, hidden_units))
        self.b1 = np.zeros((1, hidden_units), dtype=float)
        self.W2 = rng.normal(0.0, np.sqrt(2.0 / hidden_units), size=(hidden_units, out_classes))
        self.b2 = np.zeros((1, out_classes), dtype=float)

    def forward(self, X: np.ndarray):
        Z1 = X @ self.W1 + self.b1
        A1 = relu(Z1)
        Z2 = A1 @ self.W2 + self.b2
        A2 = softmax(Z2)
        cache = {"X": X, "Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
        return A2, cache

    def backward(self, cache: dict, y_true: np.ndarray):
        X, Z1, A1, A2 = cache["X"], cache["Z1"], cache["A1"], cache["A2"]
        m = X.shape[0]
        y_oh = one_hot(y_true, self.out_classes)
        dZ2 = (A2 - y_oh) / m
        dW2 = A1.T @ dZ2
        db2 = np.sum(dZ2, axis=0, keepdims=True)
        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * relu_grad(Z1)
        dW1 = X.T @ dZ1
        db1 = np.sum(dZ1, axis=0, keepdims=True)
        grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
        return grads

    def step(self, grads: dict):
        self.W1 -= self.lr * grads["dW1"]
        self.b1 -= self.lr * grads["db1"]
        self.W2 -= self.lr * grads["dW2"]
        self.b2 -= self.lr * grads["db2"]


In [ ]:
# Обучение MLP
mlp = MLP(in_features=X_train2_s.shape[1], hidden_units=8, out_classes=num_classes, lr=0.05, seed=42)

epochs = 300
loss_hist = []
acc_hist = []

for ep in range(epochs):
    probs, cache = mlp.forward(X_train2_s)
    loss = cross_entropy(one_hot(y_train2, num_classes), probs)
    grads = mlp.backward(cache, y_train2)
    mlp.step(grads)

    # метрики
    y_pred_train = probs.argmax(axis=1)
    acc_train = (y_pred_train == y_train2).mean()
    loss_hist.append(loss)
    acc_hist.append(acc_train)

fig = px.line(y=loss_hist, labels={"x": "Эпоха", "y": "Loss (CE)"}, title="MLP: кросс-энтропия по эпохам")
fig.show()
fig = px.line(y=acc_hist, labels={"x": "Эпоха", "y": "Accuracy (train)"}, title="MLP: Accuracy (train)")
fig.show()

# Оценка на test
probs_test, _ = mlp.forward(X_test2_s)
y_pred_test = probs_test.argmax(axis=1)
acc_test = (y_pred_test == y_test2).mean()
print(f"MLP точность (test): {acc_test:.3f}")


In [ ]:
# Матрица ошибок и визуализация границы для 2D

# Конфьюжен-матрица
classes = [k for k, v in sorted(label_to_id.items(), key=lambda kv: kv[1])]
cm = np.zeros((num_classes, num_classes), dtype=int)
for yt, yp in zip(y_test2, y_pred_test):
    cm[yt, yp] += 1

fig = px.imshow(cm, text_auto=True, x=classes, y=classes, color_continuous_scale='Blues', title='Confusion matrix (test)')
fig.update_xaxes(title_text='Предсказанный класс')
fig.update_yaxes(title_text='Истинный класс')
fig.show()

# Визуализация границы решения для 2D признаков
xx, yy = np.meshgrid(
    np.linspace(X_train2_s[:, 0].min()-0.5, X_train2_s[:, 0].max()+0.5, 220),
    np.linspace(X_train2_s[:, 1].min()-0.5, X_train2_s[:, 1].max()+0.5, 220)
)

grid2 = np.c_[xx.ravel(), yy.ravel()]
probs_grid, _ = mlp.forward(grid2)
zz = probs_grid.argmax(axis=1).reshape(xx.shape)

fig = go.Figure()
fig.add_trace(go.Contour(
    x=xx[0], y=yy[:, 0], z=zz,
    showscale=False, opacity=0.3,
    contours=dict(coloring='heatmap')
))

fig.add_trace(go.Scatter(
    x=X_train2_s[:, 0], y=X_train2_s[:, 1], mode='markers',
    marker=dict(color=y_train2, colorscale='Viridis', line=dict(width=0.5, color='black')),
    name='Train'
))
fig.update_layout(title='MLP: границы классов (2D)', xaxis_title=feat_mlp[0], yaxis_title=feat_mlp[1])
fig.show()


## Выводы

- Персептрон хорошо справляется с бинарной линейно-разделимой задачей; мы увидели его правило обучения и границу решения.
- Простая FCNN (ReLU + Softmax) с кросс-энтропией и backprop способна решать многоклассовую классификацию на Iris, обучаясь на NumPy.
- Важные практические моменты: стандартизация признаков, корректная инициализация весов (He для ReLU), контроль скорости обучения.

## Полезные ссылки
- Iris (описание): `https://archive.ics.uci.edu/ml/datasets/iris`
- CSV Iris: `https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv`
- Персептрон (RU): `https://ru.wikipedia.org/wiki/Персептрон`
- Обратное распространение (RU): `https://ru.wikipedia.org/wiki/Обратное_распространение_ошибки`
- CS231n Backprop: `https://cs231n.github.io/optimization-2/`
- Michael Nielsen (гл.1): `http://neuralnetworksanddeeplearning.com/chap1.html`
